In [32]:
import random
 
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

import keras.utils
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop


In [33]:
path_to_file = keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# we increase peformance  by using only lowercase
text = open(path_to_file, 'rb').read().decode(encoding='utf-8').lower()

In [34]:
# we only use this much because we dont have the necessary computational power to train the entire dataset


# sort basically sorts it and set basically just saves it as unique text.
characters = sorted(set(text))

# enumerate iterates over a list and returns the list item with an index for example -> 1, fruit
char_to_index = dict((c,i) for i, c in enumerate(characters))
index_to_char = dict((i,c) for i, c in enumerate(characters)) 

print(char_to_index)
print(index_to_char)


{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22, 'k': 23, 'l': 24, 'm': 25, 'n': 26, 'o': 27, 'p': 28, 'q': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'a', 14: 'b', 15: 'c', 16: 'd', 17: 'e', 18: 'f', 19: 'g', 20: 'h', 21: 'i', 22: 'j', 23: 'k', 24: 'l', 25: 'm', 26: 'n', 27: 'o', 28: 'p', 29: 'q', 30: 'r', 31: 's', 32: 't', 33: 'u', 34: 'v', 35: 'w', 36: 'x', 37: 'y', 38: 'z'}


In [36]:
seq_length = 40
step_size = 3

# basically sentences and next_characters are the X & Y
# which means sentences are the Input data and the next_characters are the target data

sentences = []
next_characters = []

for i in range(0, len(text) - seq_length, step_size):
    sentences.append(text[i:i+seq_length])
    next_characters.append(text[i+seq_length])

In [37]:
X = np.zeros((len(sentences), seq_length, len(characters)), dtype=bool)
Y = np.zeros((len(sentences), len(characters)), dtype=bool)


for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        X[i, t, char_to_index[character]] = 1
    Y[i, char_to_index[next_characters[i]]] = 1 

X    

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False,  True, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [38]:
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [49]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
model.fit(X, Y, batch_size=256, epochs=4)



Epoch 1/4
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 81s 55ms/step - loss: 1.3749
Epoch 2/4
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 85s 59ms/step - loss: 1.3650
Epoch 3/4
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 80s 55ms/step - loss: 1.3502
Epoch 4/4
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 78s 54ms/step - loss: 1.3462


In [51]:
model.save('textgen-s.keras')

In [52]:
model = load_model('textgen-s.keras')

In [62]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)/temperature
    exp_preds = np.exp(preds)
    preds = exp_preds/ np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - seq_length - 1)
    generated = ''
    sentence = text[start_index: start_index + seq_length]
    generated += sentence

    for i in range(length):
        x = np.zeros((1, seq_length, len(characters)))
        for t, character in enumerate(sentence):
            x[0, t, char_to_index[character]] = 1

        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_character = index_to_char[next_index] 

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated   
       

In [63]:
print('----300,0.2----')
print(generate_text(300, 0.2))
print('----500,0.2----')
print(generate_text(500, 0.2))
print('----600,0.2----')
print(generate_text(600, 0.2))

----300,0.2----
hard ii:
twice for one step i'll groan, the state,
and the throng the prince that the world,
the state of the state of the stander than it is a prison
which i have the state to the marries and the sea
the world as i would see the world the man that i have the brother that i see
the brother will not stay the streep to see
the seas, and mar
----500,0.2----
ome.
here comes his mother.

sicinius:
let me some promised the prince of the world.

provost:
i will be some more as offence the seat,
and the world the state of the seather souls,
that i see the brother and some son, and the marries,
the cause of the world so many the world.

brutus:
what shall she were the seas,
and which i have done the stander with the prince:
i have the brother than the man the greaterous son,
and the dear the staught of the seather,
but the stander than the world in the world,
and the man that i have the world 
----600,0.2----
d and would have married her perforce
to see the state of the king and